In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.day_to_day import S_driver_opt_out_TS, S_traveller_opt_out_TS
from MaaSSim.day_to_day import d2d_kpi_veh, d2d_kpi_pax

In [4]:
import osmnx as ox
import networkx as nx
import random
import math
import pandas as pd
import matplotlib.pyplot as plt          
import numpy as np
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

In [5]:
params = MaaSSim.utils.get_config('glance.json')
params.city = "Amsterdam, Netherlands"
params.paths.G = "../../data/graphs/Amsterdam.graphml"
params.paths.skim = "../../data/graphs/Amsterdam.csv"
params.vehicle_fixed_positions = True
params.times.patience = 600 #second
params.dist_threshold = 20000 #meter
params.dist_threshold_min = 3000 #meter

params.d2d.probabilistic = True
params.d2d.heterogeneous = False
params.d2d.B_income = 1
params.d2d.fuel_cost = 0.25 #euro/km

params.d2d.B_inveh_time = 1
params.d2d.B_exp_time = 2 #1.5 in oded's paper
params.PT_fare = 1 ## euro
params.PT_avg_speed = 6

params.nPM = 2 # number of platforms. platform id starts from 1
params.platforms.base_fare = 1.4 #euro
params.platforms.fare = float(1.20) #euro/km
params.platforms.min_fare = float(2) # euro
params.platforms.comm_rate = float(0.20)
params.platforms.discount = float(0.0)
params.platforms.daily_marketing = False

params.d2d.learning_d = 1 
params.d2d.B_Experience = 0.80
params.d2d.B_WOM = 0.20
params.d2d.B_Marketing = 0.0
params.d2d.diffusion_speed = 10/100 # speed for M is twice greater than WOM.
params.d2d.m = 5
params.d2d.mn = 100/12 # for NL
params.d2d.ini_att = 0.01
params.d2d.Eini_att = 0.01
params.d2d.adj_s = 2
params.VoT = 10.63 # value of time per hour
params.assert_me = False
params.seed = 1
params.paths.requests = 'Amsterdam_requests.csv'
params.simTime = 4
params.d2d.res_wage = params.simTime*10.63 #euro

In [9]:
# params.threshold_u = 0.005
params.max_revenue = 4000 # maximum revenue with the initial fare
params.alpha = 1.0
params.initial_capital = 50000
params.expense_per_day = 500
params.random_ini_position = False
params.step_size = 0.2 # euro/km
params.min_fare, params.max_fare  = 0.0, 3.0
params.initial_fares = (1.0,1.0)

params.nD = 1000
params.turnover_interval = 50 #---------------------------------
params.nP = 2000
params.nV = 200

In [10]:
sim = MaaSSim.simulators.Try_and_Select(params=params, f_driver_out=S_driver_opt_out_TS,f_trav_out=S_traveller_opt_out_TS,
                                  kpi_veh=d2d_kpi_veh,kpi_pax=d2d_kpi_pax, logger_level=logging.WARNING)

This simulation uses albatros data
20-12-24 12:28:29-WARNING-Setting up 4h simulation at 2024-12-20 10:27:59 for 200 vehicles and 2000 passengers in Amsterdam, Netherlands
============ P1 TURN ============
20-12-24 12:28:46-WARNING-simulation time 9.5 s
Day =  0 --- on the Left cell with fare 0.8
(np1,vp1) =  (95, 9)     (np2,vp2) =  (103, 13)
--------------------------------------------
20-12-24 12:29:07-WARNING-simulation time 11.1 s
Day =  1 --- on the Left cell with fare 0.8
(np1,vp1) =  (115, 17)     (np2,vp2) =  (110, 12)
--------------------------------------------
20-12-24 12:29:25-WARNING-simulation time 8.8 s
Day =  2 --- on the Left cell with fare 0.8
(np1,vp1) =  (105, 7)     (np2,vp2) =  (123, 10)
--------------------------------------------
20-12-24 12:29:40-WARNING-simulation time 8.8 s
Day =  3 --- on the Left cell with fare 0.8
(np1,vp1) =  (119, 15)     (np2,vp2) =  (106, 9)
--------------------------------------------
20-12-24 12:29:55-WARNING-simulation time 8.0 s
D

KeyboardInterrupt: 

In [9]:
sim.fare_trajectory

[(1.0, 1.0),
 (1.0, 1.2),
 (1.2, 1.2),
 (1.2, 1.4),
 (1.4, 1.4),
 (1.4, 1.6),
 (1.6, 1.6),
 (1.6, 1.8),
 (1.8, 1.8),
 (1.8, 2.0),
 (2.0, 2.0),
 (2.0, 2.2),
 (2.2, 2.2),
 (2.2, 2.4),
 (2.4, 2.4),
 (2.4, 2.6),
 (2.6, 2.6),
 (2.6, 2.8),
 (2.8, 2.8),
 (2.8, 3.0),
 (3.0, 3.0)]

In [21]:
df = sim.res[700].pax_exp
df[df.P1_EXPERIENCE_U>0.01]

,P_U,PT_U,ACTUAL_WT,U_dif,OUT,mu,wu,nDAYS_HAILED,nDAYS_TRY,P1_EXPERIENCE_U,...,ACCEPTS_OFFER,REJECTS_OFFER,ARRIVES_AT_PICKUP,MEETS_DRIVER_AT_PICKUP,DEPARTS_FROM_PICKUP,ARRIVES_AT_DROPOFF,SETS_OFF_FOR_DEST,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,LOSES_PATIENCE
pax,,,,,,,,,,,,,,,,,,,,,
22,-13.104722,-11.758797,0.000000,0.000000,True,0.0,0.0,369.0,371.0,0.030285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,-11.917658,-11.106069,0.000000,0.000000,True,0.0,0.0,503.0,503.0,0.478814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,-12.649278,-11.014238,0.000000,0.000000,True,0.0,0.0,333.0,334.0,0.030422,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,-11.433261,-10.717607,0.000000,0.000000,True,0.0,0.0,310.0,312.0,0.129899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,-10.253997,-9.667505,0.000000,0.000000,True,0.0,0.0,369.0,371.0,0.042889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,-11.070228,-10.857091,0.000000,0.000000,True,0.0,0.0,349.0,350.0,0.082980,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1908,-10.964817,-10.535015,0.000000,0.000000,True,0.0,0.0,414.0,415.0,0.400875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1918,-14.202328,-12.654704,0.000000,0.000000,True,0.0,0.0,459.0,461.0,0.187349,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Simulation Runs:
    1- Variying alpha (0.0,0.2,0.4,0.6,0.8,1.0)
    2- Minimum wage impact (with the fixed alpha)

In [1]:
sim.competition_trajectory['P2']

NameError: name 'sim' is not defined

In [10]:
params.nD = len(sim.res)

df_s = pd.DataFrame()
df_d = pd.DataFrame()

for d in range(0,params.nD):
    
    veh_exp = sim.res[d].veh_exp
    pax_exp = sim.res[d].pax_exp
    plat_exp = sim.res[d].plat_exp
    platforms = sim.res[d].platforms
    
    df_d.at[d, 'P1_EXPERIENCE_U'] = pax_exp.P1_EXPERIENCE_U.mean()
    df_d.at[d, 'P2_EXPERIENCE_U'] = pax_exp.P2_EXPERIENCE_U.mean()
    df_d.at[d, 'P1_WOM_U'] = pax_exp.P1_WOM_U.mean()
    df_d.at[d, 'P2_WOM_U'] = pax_exp.P2_WOM_U.mean()
    df_d.at[d, 'P1_MARKETING_U'] = pax_exp.P1_MARKETING_U.mean()
    df_d.at[d, 'P2_MARKETING_U'] = pax_exp.P2_MARKETING_U.mean()
    df_d.at[d, 'P1_OUT'] = pax_exp[pax_exp.platform_id==1].OUT.value_counts().get(False, 0)
    df_d.at[d, 'P2_OUT'] = pax_exp[pax_exp.platform_id==2].OUT.value_counts().get(False, 0)
    df_d.at[d, 'P1_ACTUAL_WT'] = pax_exp[pax_exp.platform_id==1].ACTUAL_WT.mean()
    df_d.at[d, 'P2_ACTUAL_WT'] = pax_exp[pax_exp.platform_id==2].ACTUAL_WT.mean()
    df_d.at[d, 'P1_MATCHING_T'] = pax_exp[pax_exp.platform_id==1].MATCHING_T.mean()
    df_d.at[d, 'P2_MATCHING_T'] = pax_exp[pax_exp.platform_id==2].MATCHING_T.mean()
    df_d.at[d, 'P1_PICKUP_T'] = veh_exp[veh_exp.platform_id==1].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.platform_id==1].nRIDES.mean()*60)
    df_d.at[d, 'P2_PICKUP_T'] = veh_exp[veh_exp.platform_id==2].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.platform_id==2].nRIDES.mean()*60)
    df_d.at[d, 'PICKUP_T'] = veh_exp[veh_exp.OUT==False].ARRIVES_AT_PICKUP.mean()/(veh_exp[veh_exp.OUT==False].nRIDES.mean()*60)
    
    df_d.at[d, 'P1_fare'] = plat_exp.fare[1]; df_d.at[d, 'P2_fare'] = plat_exp.fare[2]
    df_d.at[d, 'P1_revenue'] = plat_exp.revenue[1]; df_d.at[d, 'P2_revenue'] = plat_exp.revenue[2]
    df_d.at[d, 'P1_remaining_capital'] = plat_exp.remaining_capital[1]; df_d.at[d, 'P2_remaining_capital'] = plat_exp.remaining_capital[2]
    
    df_s.at[d, 'P1_EXPERIENCE_U'] = veh_exp.P1_EXPERIENCE_U.mean()
    df_s.at[d, 'P2_EXPERIENCE_U'] = veh_exp.P2_EXPERIENCE_U.mean()
    df_s.at[d, 'P1_WOM_U'] = veh_exp.P1_WOM_U.mean()
    df_s.at[d, 'P2_WOM_U'] = veh_exp.P2_WOM_U.mean()
    df_s.at[d, 'P1_MARKETING_U'] = veh_exp.P1_MARKETING_U.mean()
    df_s.at[d, 'P2_MARKETING_U'] = veh_exp.P2_MARKETING_U.mean()
    df_s.at[d, 'P1_OUT'] = veh_exp[veh_exp.platform_id==1].OUT.value_counts().get(False, 0)
    df_s.at[d, 'P2_OUT'] = veh_exp[veh_exp.platform_id==2].OUT.value_counts().get(False, 0)
    df_s.at[d, 'P1_IDLE_TIME'] = veh_exp[veh_exp.platform_id==1].IDLE_TIME.mean()/60
    df_s.at[d, 'P2_IDLE_TIME'] = veh_exp[veh_exp.platform_id==2].IDLE_TIME.mean()/60
    df_s.at[d, 'P1_REVENUE'] = veh_exp[veh_exp.platform_id==1].REVENUE.mean()
    df_s.at[d, 'P2_REVENUE'] = veh_exp[veh_exp.platform_id==2].REVENUE.mean()
    df_s.at[d, 'P1_ACTUAL_INC'] = veh_exp[veh_exp.platform_id==1].ACTUAL_INC.mean()
    df_s.at[d, 'P2_ACTUAL_INC'] = veh_exp[veh_exp.platform_id==2].ACTUAL_INC.mean()
    df_s.at[d, 'P1_COST'] = veh_exp[veh_exp.platform_id==1].COST.mean()
    df_s.at[d, 'P2_COST'] = veh_exp[veh_exp.platform_id==2].COST.mean()
    
    if d==0: d_cols = df_d.columns.values.tolist()
    if d==0: s_cols = df_s.columns.values.tolist()
    
    ld = sim.res[0].pax_exp.columns.values.tolist()
    ldn = [i for i in ld if i not in d_cols]
    for col in ldn:
        df_d.at[d, col] = pax_exp[pax_exp.OUT==False][col].mean()
        
    ls = sim.res[0].veh_exp.columns.values.tolist()
    lsn = [i for i in ls if i not in s_cols]
    for col in lsn:
        df_s.at[d, col] = veh_exp[veh_exp.OUT==False][col].mean()
        
run_id = '1.0_1_1'
df_d.to_csv('csv/1.1/demand_{}.csv'.format(run_id))
df_s.to_csv('csv/1.1/supply_{}.csv'.format(run_id))